# ens002

In [1]:
import os
import sys
import gc
import itertools
import pickle
import pathlib
import datetime
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv
load_dotenv()
sys.path.append(os.getenv('UTILS_PATH'))

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.optimize import minimize

import line_notify

In [2]:
import builtins
import types

def imports():
    for name, val in globals().items():
        # module imports
        if isinstance(val, types.ModuleType):
            yield name, val

            # functions / callables
        if hasattr(val, '__call__'):
            yield name, val


def noglobal(f):
    '''
    ref: https://gist.github.com/raven38/4e4c3c7a179283c441f575d6e375510c
    '''
    return types.FunctionType(f.__code__,
                              dict(imports()),
                              f.__name__,
                              f.__defaults__,
                              f.__closure__
                              )

ディレクトリ設定

In [3]:
INPUT_DIR = os.getenv('INPUT_DIR')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')
#exp_name = os.path.dirname(__file__).split('/')[-1]
exp_name = 'ens001'
os.makedirs(OUTPUT_DIR + exp_name, exist_ok=True)

In [4]:
SEED = 48
exps = ['exp035', 'exp038', 'exp043', 'exp044', 'exp045', 'exp050', 'exp055']

データセット準備

In [5]:
articles = pd.read_csv(INPUT_DIR + 'articles.csv', dtype='object')
customers = pd.read_csv(INPUT_DIR + 'customers.csv')
transactions = pd.read_csv(INPUT_DIR + 'transactions_train.csv', dtype={'article_id':'str'}, parse_dates=['t_dat'])
sample = pd.read_csv(INPUT_DIR + 'sample_submission.csv')

ALL_CUSTOMER = customers['customer_id'].unique().tolist()
ALL_ARTICLE = articles['article_id'].unique().tolist()

customer_ids = dict(list(enumerate(ALL_CUSTOMER)))
article_ids = dict(list(enumerate(ALL_ARTICLE)))

customer_map = {u: uidx for uidx, u in customer_ids.items()}
article_map = {i: iidx for iidx, i in article_ids.items()}

articles['article_id'] = articles['article_id'].map(article_map)
customers['customer_id'] = customers['customer_id'].map(customer_map)
transactions['article_id'] = transactions['article_id'].map(article_map)
transactions['customer_id'] = transactions['customer_id'].map(customer_map)
sample['customer_id'] = sample['customer_id'].map(customer_map)

In [6]:
# 正解データ
valid_start = '2020-09-16'
valid_end = '2020-09-22'
valid = transactions[(transactions['t_dat'] >= valid_start) & (transactions['t_dat'] <= valid_end)].copy()
valid = valid[['customer_id', 'article_id']].drop_duplicates()
valid = valid.groupby('customer_id')['article_id'].apply(list).reset_index()
valid = valid.sort_values('customer_id').reset_index(drop=True)

In [7]:
# OOFデータ
oof = pd.DataFrame()
for exp in exps:
    exp_oof = pd.read_csv(OUTPUT_DIR + f'{exp}/{exp}_oof.csv', usecols=['customer_id', 'article_id', 'pred'])
    exp_oof = exp_oof.sort_values(['customer_id', 'pred'], ascending=False)
    exp_oof = exp_oof.groupby('customer_id').head(12)
    exp_oof = exp_oof.sort_values(['customer_id', 'pred'])
    exp_oof['exp'] = exp
    exp_oof['rank'] = exp_oof.groupby('customer_id').cumcount() + 1
    oof = pd.concat([oof, exp_oof[['customer_id', 'article_id', 'exp', 'rank']]])

In [8]:
# TESTデータ
def func(s, idx):
    s = s.split()
    return s[idx]

test = pd.DataFrame()
for exp in exps:
    exp_test = pd.read_csv(OUTPUT_DIR + f'{exp}/{exp}_sub.csv')
    prediction = exp_test['prediction']

    for i in range(12):
        exp_test_tmp = exp_test[['customer_id']]
        exp_test_tmp['exp'] = exp
        exp_test_tmp['article_id'] = prediction.apply(func, idx=i)
        exp_test_tmp['rank'] = 12 - i
        test = pd.concat([test, exp_test_tmp])

In [9]:
def apk(y_true, y_pred, K=12):
    assert(len(y_true) == len(y_pred))
    apks = []
    for idx in range(len(y_true)):
        y_i_true = y_true[idx]
        y_i_pred = y_pred[idx]

        # 予測値の数と重複の確認
        assert(len(y_i_pred) <= K)
        assert(len(np.unique(y_i_pred)) == len(y_i_pred))

        sum_precision = 0.0
        num_hits = 0.0

        for i, p in enumerate(y_i_pred):
            if p in y_i_true:
                num_hits += 1
                precision = num_hits / (i+1)
                sum_precision += precision
        apk = sum_precision / min(len(y_i_true), K)
        apks.append(apk)
    return apks

In [10]:
def f(x):
    oof_ = oof.copy()
    oof_['w'] = 0
    for exp, w in zip(exps, x):
        oof_.loc[oof_['exp']==exp, 'w'] = w
    oof_['rank'] = oof_['rank'] * oof_['w']
    oof_ = oof_.sort_values(['customer_id', 'rank'], ascending=False)
    oof_ = oof_.drop_duplicates(subset=['customer_id', 'article_id'])
    oof_ = oof_.groupby('customer_id').head(12)
    oof_ = oof_.groupby('customer_id')['article_id'].apply(list).reset_index()
    oof_ = oof_.sort_values('customer_id').reset_index(drop=True)
    score = np.mean(apk(valid['article_id'].tolist(), oof_['article_id'].tolist()))
    print(-score)
    return -score

In [11]:
init_state = np.ones(len(exps))
result = minimize(f, init_state, method='Nelder-Mead')

-0.03261878377896062
-0.03261878377896062
-0.032946388189571144
-0.03260682138028434
-0.03302429131744699
-0.03310902080035748
-0.032885538552568615
-0.03297856630317894
-0.0326427026256682
-0.03306006582779279
-0.03305373123405375
-0.03269938677313558
-0.03269938677313558
-0.03318336194852103
-0.0326427026256682
-0.03305332831124471
-0.0331182087118085
-0.033189640672561846
-0.03322133463791598
-0.03323638584325903
-0.03324329495375699
-0.03324377545195901
-0.03304057194628101
-0.03323734339045209
-0.03320000755591912
-0.03323966972844216
-0.033221711979047025
-0.03323921752190872
-0.03327551555870941
-0.033266319673925744
-0.03326166325102034
-0.033269642145210075
-0.0332714723264258
-0.03311693650846492
-0.03325628042031621
-0.03326719283002848
-0.03325917173952953
-0.033280586445460836
-0.03326986621220607
-0.03329921157491352
-0.03330703234523037
-0.03335162548084035
-0.033353268195600666
-0.033281301778197055
-0.03334033661027375
-0.03328148968519643
-0.033297802328363456
-0.0333

In [12]:
print('optimized CV: ', -result['fun'])
print('w: ', result['x'])

optimized CV:  0.03342350577408703
w:  [0.59017257 1.32194816 0.57782268 0.98713707 1.39865446 0.93933254
 1.37225142]


# sub作成

In [13]:
ws = list(result['x'])
test['w'] = 0
for exp, w in zip(exps, ws):
    test.loc[test['exp']==exp, 'w'] = w
    test['rank'] = test['rank'] * test['w']

test = test.sort_values(['customer_id', 'rank'], ascending=False)
test = test.drop_duplicates(subset=['customer_id', 'article_id'])
test = test.groupby('customer_id').head(12)
test = test.groupby('customer_id')['article_id'].apply(list).reset_index()

sub = sample['customer_id'].map(customer_ids).to_frame()
sub = sub.merge(test, on=['customer_id'], how='left')
sub = sub.rename(columns={'article_id':'prediction'})
assert(sub['prediction'].apply(len).min()==12)
sub['prediction'] = sub['prediction'].apply(lambda x: ' '.join(x))
sub.to_csv(OUTPUT_DIR + f'{exp_name}/{exp_name}_sub.csv', index=False)

In [14]:
sub

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601006 0568601043 0915529003 0924243002 06...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0714790020 0448509014 0915529003 0706016001 09...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0901666001 0924243001 0794321008 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0751471043 0918522001 0918292001 0791587001 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0896152002 0730683050 0924243002 0791587015 06...
...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,0557599022 0866731001 0720125039 0918292001 08...
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0762846008 0706016001 0448509014 0706016003 07...
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,0762846027 0706016002 0866731001 0918292001 07...
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,0714790020 0448509014 0706016002 0706016001 07...
